Introduction

Import statements

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
here = os.getcwd()

afilename= F"{here}/data/2020/map-keys-2020.csv"
map_keys = pd.read_csv(afilename)

afilename = here+"/data/2020/species2020_n23.csv"
samples = pd.read_csv(afilename)

In [3]:
samples.head()

,place,date,species,name,slug
0,alleestrasse-1,2020-09-02,plantago lanceolata,plantain lancéolé,plantago-lanceolata
1,alleestrasse-1,2020-09-02,centaurea nigra,centaurée noire,centaurea-nigra
2,alleestrasse-1,2020-09-02,plantago media,plantain moyen,plantago-media
3,alleestrasse-1,2020-09-02,chenopodium album,chénopode blanc,chenopodium-album
4,alleestrasse-1,2020-09-02,centaurea jacea,centaurée jacée,centaurea-jacea


In [4]:
map_keys.head()

,lon,lat,place
0,7.253285,47.138823,jura-2
1,7.791570,46.685610,sundgraben-port
2,7.252867,47.137585,salome-3
3,7.252760,47.137273,salome-4
4,7.252618,47.136619,alleestrasse-4


In [5]:
samples.columns

Index(['place', 'date', 'species', 'name', 'slug'], dtype='object')

In [6]:
samples['loc_date'] = list(zip(samples.place, samples.date))
# datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
def check_dates(x):
    try:
        data = dt.datetime.strptime(x, "%Y-%m-%d")
    except:
        data = 'no date'
    return data
samples['date_stamp'] = samples['date'].map(lambda x: check_dates(x))
samples['month'] = samples.date_stamp.dt.month

print(F"There are {len(samples[samples.date_stamp == 'no date'])} records with no valid date")
print(F"\nThese are the months of the year that sampling occurred:\n{samples.month.unique()}\n")

There are 0 records with no valid date

These are the months of the year that sampling occurred:
[9 7 6 8]



In [7]:
som_samples = samples.groupby(['loc_date', 'species']).species.count()

print(F"\nThe species count for one sample alleestrasse-1, 2020-09-02:\n\n{som_samples.loc[('alleestrasse-1', '2020-09-02')]}")
# som_samples.loc[("alleestrasse-1", "2020-09-02")]


The species count for one sample alleestrasse-1, 2020-09-02:

species
centaurea jacea        1
centaurea nigra        1
chenopodium album      1
geranium pyrenaicum    1
plantago lanceolata    1
plantago media         1
veronica persica       1
Name: species, dtype: int64


In [8]:
som_locations = samples.groupby(['place', 'species']).species.count()

print(F"\nThe species count for one location alleestrasse-1:\n\n{som_locations.loc['alleestrasse-1']}\n")


The species count for one location alleestrasse-1:

species
centaurea jacea        3
centaurea nigra        1
chenopodium album      1
daucus carota          1
geranium pyrenaicum    1
plantago lanceolata    1
plantago media         1
veronica persica       1
Name: species, dtype: int64



In [9]:
# all the records from one location:

allseestrasseone = samples[samples.place == 'alleestrasse-1']

print(F"\nAll the samples from one location and the number of times a species was identified:\n\n{allseestrasseone.groupby(['loc_date', 'species']).name.count()}\n")


All the samples from one location and the number of times a species was identified:

loc_date                      species            
(alleestrasse-1, 2020-07-28)  centaurea jacea        2
                              daucus carota          1
(alleestrasse-1, 2020-09-02)  centaurea jacea        1
                              centaurea nigra        1
                              chenopodium album      1
                              geranium pyrenaicum    1
                              plantago lanceolata    1
                              plantago media         1
                              veronica persica       1
Name: name, dtype: int64



In [10]:
species_list = samples.species.unique()

species_group = samples.groupby(['species', 'place']).loc_date.count()

print(F"The number of times and places a species was identified:\n\n{species_group.loc['vicia cracca']}\n")

The number of times and places a species was identified:

place
chapelle         1
gottstatt-3      1
twann-estuary    1
Name: loc_date, dtype: int64

